<a href="https://colab.research.google.com/github/kauagomess1/imers-o-alura-google/blob/main/imersaoaluragoogle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
# -*- coding: utf-8 -*-
"""Célula 1: Configuração Inicial e Instalação"""

# Instalando as bibliotecas necessárias: google-genai (SDK básico) e google-adk (Framework de Agentes)
print("Célula 1: Instalando bibliotecas google-genai e google-adk...")
# O -q instala silenciosamente.
%pip -q install google-genai google-adk
print("Instalação completa.")

# Importando as bibliotecas necessárias para setup, API e ADK
import os # Para configurar variáveis de ambiente
from google.colab import userdata # Para carregar a API Key de forma segura
import google.generativeai as genai # SDK do Gemini (usado internamente pelo ADK Runner/Agent)
from google.adk.agents import Agent # Classe base para definir um Agente
from google.adk.runners import Runner # Classe para executar um Agente
from google.adk.sessions import InMemorySessionService # Serviço de sessão simples para o Runner
from google.genai import types # Para criar o formato de conteúdo que o ADK usa
import warnings # Para ignorar avisos

# Configura a API Key do Google Gemini
print("\nCélula 1: Configurando a API Key...")
try:
    # Certifique-se de ter sua API Key armazenada em Colab Secrets com o nome 'GOOGLE_API_KEY'.
    os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
    # Verifica se a chave foi carregada (opcional, mas boa prática)
    if not os.environ.get("GOOGLE_API_KEY"):
         # Se a chave não carregar, dispara um erro para parar a execução.
         raise ValueError("API Key não encontrada nos Colab Secrets com o nome 'GOOGLE_API_KEY'.")
    print("API Key carregada e variável de ambiente configurada com sucesso!")
except Exception as e:
    print(f"\nERRO Crítico: Não foi possível carregar a API Key. Detalhes: {e}")
    print("Por favor, configure sua API Key nos Colab Secrets com o nome 'GOOGLE_API_KEY' e execute a célula novamente.")
    # Dispara o erro para impedir a execução do resto do código sem a chave.
    raise

# Define o modelo a ser usado pelos Agentes
MODEL_ID = "gemini-2.0-flash"

# Ignora alguns avisos de execução (estrutura do seu código original)
warnings.filterwarnings("ignore")

print(f"\nCélula 1: Setup completo. Modelo configurado: {MODEL_ID}")

Célula 1: Instalando bibliotecas google-genai e google-adk...
Instalação completa.

Célula 1: Configurando a API Key...
API Key carregada e variável de ambiente configurada com sucesso!

Célula 1: Setup completo. Modelo configurado: gemini-2.0-flash


In [65]:
# -*- coding: utf-8 -*-
"""Célula 2: Funções Auxiliares"""

# Importa bibliotecas adicionais necessárias para as funções auxiliares
import textwrap # Para formatar o texto
from IPython.display import display, Markdown # Para exibir Markdown no Colab

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
# Esta função encapsula a complexidade de rodar um Agente ADK
def call_agent(agent: Agent, message_text: str) -> str:
    """Cria uma sessão temporária e executa um Agente ADK com a mensagem fornecida."""
    # Cria um serviço de sessão em memória. Simples para este exemplo de chat único por chamada.
    session_service = InMemorySessionService()
    # Cria uma nova sessão para esta interação. Você pode personalizar os IDs se precisar de sessões persistentes.
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para executar o agente nesta sessão.
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada no formato que o ADK espera
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera pelos eventos retornados pela execução do agente.
    # O ADK gerencia a interação do agente com o modelo e ferramentas internamente.
    try:
        # O método run() executa a lógica do agente com a nova mensagem.
        for event in runner.run(user_id="user1", session_id="session1", new_message=content):
            # Verificamos se o evento é a resposta final do agente.
            if event.is_final_response():
                # Concatena as partes de texto da resposta final
                for part in event.content.parts:
                    if part.text is not None:
                        final_response += part.text
                # Quebra de linha entre partes, se houver
                if final_response and not final_response.endswith("\n"):
                     final_response += "\n"
        # Retorna a resposta final, removendo espaços extras no início/fim.
        return final_response.strip()
    except Exception as e:
         # Tratamento básico de erros durante a execução do agente.
         print(f"\n⚠️ Erro na execução do Agente '{agent.name}': {e}")
         return f"Desculpe, o Agente Tutor teve um problema técnico ao processar sua solicitação com o Agente '{agent.name}'."

# Função auxiliar para exibir texto formatado em Markdown no Colab
# Útil para formatar blocos de código, listas, etc., na saída do notebook.
def to_markdown(text):
    """Formata texto para exibição em Markdown no Colab, adicionando indentação."""
    if not text:
        return Markdown("") # Retorna um Markdown vazio se o texto for vazio
    # Substitui marcadores de lista comuns por algo que Markdown entenda (opcional)
    text = text.replace('•', '  *')
    # Adiciona indentação para destacar a resposta do agente visualmente.
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("Célula 2: Funções auxiliares 'call_agent' e 'to_markdown' definidas.")

Célula 2: Funções auxiliares 'call_agent' e 'to_markdown' definidas.


In [66]:
# -*- coding: utf-8 -*-
"""Célula 3: Definição dos Agentes Especializados e Conhecimento do Tutor"""

# Define as linguagens que nosso Agente Tutor suporta
# Usaremos esta lista para a lógica de detecção de linguagem na próxima célula.
LINGUAGENS_SUPORTADAS = ["python", "javascript", "html_css", "c++"] # html_css como uma categoria única

# Definição das Funções que Criam os Agentes Especializados
# Cada função abaixo cria e configura uma instância de Agente ADK para uma tarefa de tutoria específica.
# A "instruction" de cada agente é o prompt principal que guia o Gemini para realizar a tarefa.
# Este texto da instruction é o resultado do seu trabalho de design de prompts no Google AI Studio.

# Agente para Explicar Conceitos
def agente_explicador():
    """Cria um Agente ADK especializado em explicar conceitos de programação para iniciantes."""
    return Agent(
        name="AgenteExplicador",
        model=MODEL_ID, # Usa o modelo definido na Célula 1
        # A instruction é o prompt que o agente usará internamente com o modelo.
        # Ela combina o papel do agente (tutor) com o formato de entrada/saída esperado.
        instruction="""
        Você é um tutor de programação muito paciente e didático, focado em ajudar iniciantes.
        Sua tarefa é explicar o conceito ou responder à pergunta do usuário sobre a linguagem especificada de forma simples, usando linguagem clara e analogias se apropriado.
        Sempre forneça um pequeno exemplo de código comentado, relevante para o conceito e a linguagem.
        Considere que o usuário é um iniciante total na linguagem especificada.

        Sua entrada virá no formato "Linguagem: [linguagem]\nConteúdo: [conceito ou pergunta]".
        Com base nisso, gere a explicação e o exemplo.

        Exemplo de como você deve responder:
        Uma lista em Python é como uma caixa onde você pode guardar vários itens em ordem.
        ... explicação detalhada e simples ...
        ```python
        # Exemplo de lista em Python
        minha_lista = [1, 2, "tres", 4.5]
        print(minha_lista) # Saída: [1, 2, 'tres', 4.5]
        ```
        """,
        description="Agente que explica conceitos de programação para iniciantes."
        # Não usamos tools como Google Search neste agente básico, ele usa o conhecimento pré-treinado do modelo.
    )

# Agente para Gerar Código Exemplo
def agente_gerador_codigo():
    """Cria um Agente ADK especializado em gerar código exemplo simples para iniciantes."""
    return Agent(
        name="AgenteGeradorCodigo",
        model=MODEL_ID,
        instruction="""
        Você é um tutor de programação que ajuda iniciantes gerando exemplos de código.
        Sua tarefa é gerar um pequeno e completo exemplo de código na linguagem especificada que realize a tarefa descrita.
        O código deve ser o mais simples possível, usar apenas conceitos básicos e ser bem comentado para facilitar o aprendizado do iniciante.

        Sua entrada virá no formato "Linguagem: [linguagem]\nTarefa: [descrição da tarefa]".
        Com base nisso, gere o código de exemplo.

        Exemplo de como você deve responder:
        Aqui está um exemplo simples de como mostrar "Olá, Mundo!" no console em JavaScript:
        ```javascript
        // Este código imprime a mensagem no console do navegador ou Node.js
        console.log("Olá, Mundo!");
        ```
        """,
        description="Agente que gera exemplos de código simples para iniciantes."
    )

# Agente para Debugar Código (Identificar Erros Simples)
def agente_debugador():
    """Cria um Agente ADK especializado em identificar e explicar erros simples de código para iniciantes."""
    return Agent(
        name="AgenteDebugador",
        model=MODEL_ID, # Usa o modelo definido na Célula 1
        instruction="""
        Você é um assistente de debug para iniciantes em programação.
        Sua tarefa é analisar o código fornecido na linguagem especificada, identificar erros comuns de sintaxe ou lógica para iniciantes e explicar passo a passo como corrigir.
        Explique o *motivo* do erro de forma didática e amigável, focando em conceitos básicos.

        Sua entrada virá no formato "Linguagem: [linguagem]\nCódigo: [código a ser debugado]".
        Com base nisso, analise o código e forneça a explicação e a correção.

        Exemplo de como você deve responder:
        Olá! Parece que há alguns pequenos errinhos de iniciante neste código Python. Não se preocupe, é normal no começo!
        Vamos ver os pontos que precisam de ajuste:
        1. Depois do 'def minha_funcao()', faltou os dois pontos ':'. Em Python, eles indicam o início de um bloco de código.
        2. A linha 'print("Olá")' precisa estar indentada (com espaços ou tab) para mostrar que faz parte da função.

        Aqui está o código corrigido:
        ```python
        def minha_funcao():
            print("Olá")
        ```
        """,
        description="Agente que ajuda a debugar erros simples para iniciantes."
    )

print("Célula 3: Definição de linguagens suportadas e funções de criação dos agentes (Explicador, Gerador, Debugador) completa.")

Célula 3: Definição de linguagens suportadas e funções de criação dos agentes (Explicador, Gerador, Debugador) completa.


In [72]:
# -*- coding: utf-8 -*-
"""Célula 4: Loop Principal de Interação e Orquestração do Agente"""

from datetime import date

# --- Inicialização e Mensagens de Boas-Vindas ---

# Instancia os agentes especializados uma vez no início
explicador = agente_explicador()
gerador = agente_gerador_codigo()
debugador = agente_debugador()

print("\n🚀 Iniciando o Agente Tutor de Programação ADK 🚀")
print("Olá! Sou seu tutor de programação para iniciantes.")
print("Posso ajudar com:")
print("- Explicações de conceitos")
print("- Exemplos de código")
print("- Ajuda com erros simples (debug)")
#linguagens suportadas
print("Linguagens suportadas (neste exemplo):", ", ".join([l.replace("_", "/") for l in LINGUAGENS_SUPORTADAS]))
print("\nPor favor, diga em qual linguagem você precisa de ajuda e o que você quer:")
print("Use o formato 'Linguagem: Sua Pergunta/Pedido'")
print("Exemplos:")
print("- Python: Explique if/else")
print("- JavaScript: Mostrar código for")
print("- HTML/CSS: <div<p>oi</p>/div>") # Para debug, coloque o código após o prefixo
print("- C++: Debug loop for")
print("Digite 'sair' a qualquer momento para encerrar.")
print("-" * 60)

#Loop Principal do Agente Orquestrador

while True:

    user_input = input("Você> ")

    # Verifica se o usuário quer sair do chat
    if user_input.lower() == 'sair':
        print("\nAgente Tutor> Chat encerrado. Bons estudos!")
        break # Sai do loop while True

    # Lógica de Orquestração: Analisando a Intenção do Usuário
    # Esta é a "lógica de decisão"
    # Ele tenta identificar a linguagem e a tarefa solicitada pelo usuário.

    agente_a_chamar = None # Variável para guardar qual agente especializado será chamado
    input_para_agente = None # Variável para guardar a entrada formatada para o agente
    linguagem_detectada = None
    tarefa_detectada = None

    user_input_limpa = user_input.strip()
    input_lower = user_input_limpa.lower()

    # Itera sobre as linguagens suportadas para ver qual prefixo casa com a entrada do usuário.
    for lang in LINGUAGENS_SUPORTADAS:
        # Cria o prefixo esperado no formato "linguagem:" (ex: "python:", "js:", etc.)
        # Substitui "_" por "/" no prefixo para permitir "html/css:" na entrada.
        prefix = f"{lang.replace('_', '/')}:"
        if input_lower.startswith(prefix.lower()):
            linguagem_detectada = lang
            # Remove o prefixo da query para passar apenas a pergunta/código para o agente especializado.
            user_query_sem_prefixo = user_input_limpa[len(prefix):].strip()
            break # Encontrou a linguagem, sai do loop de busca

    # Se a linguagem não foi detectada após verificar todos os prefixos suportados
    if not linguagem_detectada:
        print("Agente Tutor> Não consegui identificar a linguagem. Por favor, comece sua mensagem com um prefixo válido (Ex: 'Python:', 'JS:', 'HTML/CSS:', ou 'C++:').")
        print("-" * 60)
        continue # Pula para a próxima iteração do loop while

    # Usa palavras-chave simples para identificar a tarefa. Esta é parte da lógica de decisão.
    if user_query_sem_prefixo.startswith("explicar"):
        tarefa_detectada = "explicar"
        agente_a_chamar = explicador # Seleciona o Agente Explicador
        # Prepara a entrada no formato que o AgenteExplicador espera ("Linguagem: ...\nConteúdo: ...")
        query_para_agente_conteudo = user_query_sem_prefixo[len('explicar'):].strip()
        input_para_agente = f"Linguagem: {linguagem_detectada}\nConteúdo: {query_para_agente_conteudo}"

    elif user_query_sem_prefixo.startswith("mostrar código") or user_query_sem_prefixo.startswith("exemplo") or user_query_sem_prefixo.startswith("gere código"):
        tarefa_detectada = "gerar_codigo"
        agente_a_chamar = gerador # Seleciona o Agente Gerador de Código
        # Prepara a entrada no formato que o AgenteGeradorCodigo espera ("Linguagem: ...\nTarefa: ...")
        # Remove a palavra-chave da tarefa ("mostrar código", "exemplo", etc.) do início da query.
        query_sem_tarefa_keyword = user_query_sem_prefixo.split(" ", 1)[1].strip() if " " in user_query_sem_prefixo else ""
        input_para_agente = f"Linguagem: {linguagem_detectada}\nTarefa: {query_sem_tarefa_keyword}"
        # Verifica se ainda há alguma descrição da tarefa após remover a palavra-chave.
        if not query_sem_tarefa_keyword:
             print(f"Agente Tutor> O que você quer que eu mostre um código em {linguagem_detectada.replace('_', '/')}. Por favor, descreva a tarefa após a palavra-chave (Ex: '{linguagem_detectada.replace('_', '/')}: Mostrar código de loop for').")
             print("-" * 60)
             continue # Pula para a próxima iteração se não houver descrição da tarefa

    elif user_query_sem_prefixo.startswith("debug") or user_query_sem_prefixo.startswith("corrigir"):
        tarefa_detectada = "debugar"
        agente_a_chamar = debugador # Seleciona o Agente Debugador
        # Prepara a entrada no formato que o AgenteDebugador espera ("Linguagem: ...\nCódigo: ...")
        # Assume que o resto da query após a palavra-chave "debug" é o código a ser depurado.
        codigo_para_debug = user_query_sem_prefixo.split(" ", 1)[1].strip() if " " in user_query_sem_prefixo else ""
        # Verifica se o usuário forneceu código para depurar.
        if not codigo_para_debug:
             print(f"Agente Tutor> Para depurar código em {linguagem_detectada.replace('_', '/')}, por favor, forneça o código após a palavra-chave (Ex: '{linguagem_detectada.replace('_', '/')}: Debug [código aqui]').")
             print("-" * 60)
             continue # Pula para a próxima iteração se não houver código
        input_para_agente = f"Linguagem: {linguagem_detectada}\nCódigo: {codigo_para_debug}"

    else:
        # Se nenhuma tarefa específica (mostrar código, debug) for detectada, assume que é para explicar.
        tarefa_detectada = "explicar"
        agente_a_chamar = explicador # Seleciona o Agente Explicador
        # Prepara a entrada para explicar (usa o resto da query como o conceito/pergunta).
        input_para_agente = f"Linguagem: {linguagem_detectada}\nConteúdo: {user_query_sem_prefixo}"

    # Se a lógica de decisão identificou um agente e preparou a entrada para ele...
    if agente_a_chamar and input_para_agente is not None:
        print(f"Agente Tutor> Hmm, pensando em como te ajudar com {linguagem_detectada.replace('_', '/')} ({tarefa_detectada})...")
        # Chama a função auxiliar (da Célula 2) que executa o agente ADK com a entrada formatada.
        gemini_response = call_agent(agente_a_chamar, input_para_agente)

        # Exibe a resposta formatada para o usuário.
        print(f"\n--- Resposta do {agente_a_chamar.name} ---")
        display(to_markdown(gemini_response))
    else:
        # Este else só deve ser alcançado se houver um problema na lógica de seleção/preparação.
         print("Agente Tutor> Desculpe, ocorreu um problema interno ao tentar processar sua solicitação.")

    print("-" * 60) # Separador para próxima interação

# --- Fim do Loop Principal ---
# Esta linha será executada quando o loop while for quebrado (ao digitar 'sair').
print("\nAgente Tutor> Chat encerrado. Continue explorando o mundo da programação!")


🚀 Iniciando o Agente Tutor de Programação ADK 🚀
Olá! Sou seu tutor de programação para iniciantes.
Posso ajudar com:
- Explicações de conceitos
- Exemplos de código
- Ajuda com erros simples (debug)
Linguagens suportadas (neste exemplo): python, javascript, html/css, c++

Por favor, diga em qual linguagem você precisa de ajuda e o que você quer:
Use o formato 'Linguagem: Sua Pergunta/Pedido'
Exemplos:
- Python: Explique if/else
- JavaScript: Mostrar código for
- HTML/CSS: <div<p>oi</p>/div>
- C++: Debug loop for
Digite 'sair' a qualquer momento para encerrar.
------------------------------------------------------------
Você> JavaScript: o que é JS?
Agente Tutor> Hmm, pensando em como te ajudar com javascript (explicar)...

--- Resposta do AgenteExplicador ---


> JavaScript é uma linguagem de programação que roda nos navegadores web e permite adicionar interatividade às páginas. Imagine que o HTML é a estrutura de uma casa e o CSS é a decoração; o JavaScript é como os aparelhos eletrônicos que fazem a casa funcionar (luzes que acendem, televisão que liga, etc.).
> 
> Com JavaScript, você pode fazer coisas como validar formulários antes de enviar, criar animações, atualizar partes da página sem precisar recarregar tudo e muitas outras coisas.
> 
> Aqui está um exemplo simples de código JavaScript que exibe uma mensagem quando um botão é clicado:
> 
> ```javascript
> // Este código será executado quando a página HTML for carregada
> document.addEventListener('DOMContentLoaded', function() {
>   // Encontra o botão no HTML pelo ID
>   const botao = document.getElementById('meuBotao');
> 
>   // Adiciona um "ouvinte" de clique ao botão
>   botao.addEventListener('click', function() {
>     // Quando o botão é clicado, esta função é executada
>     alert('Olá, mundo!'); // Mostra um alerta na tela
>   });
> });
> ```
> 
> Neste exemplo:
> 
> 1.  `document.addEventListener('DOMContentLoaded', function() { ... });` garante que o código JavaScript só seja executado depois que a página HTML estiver completamente carregada.
> 2.  `document.getElementById('meuBotao');` busca um elemento HTML que tem o `id="meuBotao"`. No HTML, você teria algo como `<button id="meuBotao">Clique aqui</button>`.
> 3.  `botao.addEventListener('click', function() { ... });` adiciona um "ouvinte" que espera por um clique no botão. Quando o clique acontece, a função dentro do `addEventListener` é executada.
> 4.  `alert('Olá, mundo!');` mostra uma caixa de mensagem na tela com o texto "Olá, mundo!".
> 
> JavaScript é uma ferramenta poderosa para tornar as páginas web mais dinâmicas e interativas!

------------------------------------------------------------
Você> HTML/CSS o que é css?
Agente Tutor> Não consegui identificar a linguagem. Por favor, comece sua mensagem com um prefixo válido (Ex: 'Python:', 'JS:', 'HTML/CSS:', ou 'C++:').
------------------------------------------------------------
Você> HTML/CSS: o que é css?
Agente Tutor> Hmm, pensando em como te ajudar com html/css (explicar)...

--- Resposta do AgenteExplicador ---


> CSS é como o maquiador do HTML. Imagine que o HTML é uma pessoa que você quer apresentar ao mundo. O HTML cuida da estrutura dessa pessoa: qual a altura dela, qual o formato do rosto, etc. Mas ele não diz nada sobre a roupa, a cor do cabelo, a maquiagem. É aí que entra o CSS.
> 
> O CSS cuida da aparência do HTML. Ele define a cor das coisas, o tamanho das letras, o espaçamento entre os elementos, o posicionamento de cada coisa na tela. Com o CSS, você pode transformar um HTML básico em algo bonito e atraente.
> 
> ```html
> <!DOCTYPE html>
> <html>
> <head>
> <title>Exemplo de CSS</title>
> <style>
> /* Isso é um comentário em CSS */
> body {
>   background-color: lightblue; /* Define a cor de fundo da página */
> }
> 
> h1 {
>   color: white; /* Define a cor do texto do título */
>   text-align: center; /* Centraliza o texto do título */
> }
> 
> p {
>   font-family: verdana; /* Define a fonte do texto do parágrafo */
>   font-size: 20px; /* Define o tamanho da fonte do parágrafo */
> }
> </style>
> </head>
> <body>
> 
> <h1>Meu Primeiro Título em CSS</h1>
> <p>Este é um parágrafo.</p>
> 
> </body>
> </html>
> ```
> 
> Neste exemplo:
> 
> -   `body { background-color: lightblue; }` define que a cor de fundo da página toda será azul claro.
> -   `h1 { color: white; text-align: center; }` define que todos os títulos (`<h1>`) terão a cor branca e o texto centralizado.
> -   `p { font-family: verdana; font-size: 20px; }` define que todos os parágrafos (`<p>`) usarão a fonte "verdana" e terão tamanho 20.
> 
> O CSS é uma ferramenta poderosa para deixar suas páginas web mais bonitas e organizadas!

------------------------------------------------------------
Você> c++: me explique como usar em jogos na Unreal Engine?
Agente Tutor> Hmm, pensando em como te ajudar com c++ (explicar)...

--- Resposta do AgenteExplicador ---


> Em C++, usar a Unreal Engine para criar jogos envolve entender como a engine organiza o código e como você pode estender suas funcionalidades. Imagine a Unreal Engine como um kit de construção gigante, onde você usa peças (classes e funções) pré-fabricadas para montar seu jogo. O C++ serve como a cola e as instruções para personalizar essas peças e criar novas.
> 
> **Conceitos Fundamentais:**
> 
> 1.  **Classes:** Na Unreal Engine, tudo é baseado em classes. Classes são como "moldes" para criar objetos. Por exemplo, você tem a classe `Actor` que representa qualquer objeto no mundo do jogo, como um personagem, uma porta ou uma árvore.
> 2.  **Herança:** A herança permite criar novas classes baseadas em classes existentes, herdando características e adicionando ou modificando comportamentos. Por exemplo, você pode criar uma classe `MeuPersonagem` que herda de `Character` (uma classe da Unreal para personagens) e adicionar sua própria lógica de movimento e habilidades.
> 3.  **Componentes:** Componentes são como "peças" que você adiciona a um `Actor` para dar a ele funcionalidades. Por exemplo, um `StaticMeshComponent` permite que um `Actor` exiba um modelo 3D, enquanto um `AudioComponent` permite que ele toque sons.
> 4.  **Funções:** Funções são blocos de código que realizam tarefas específicas. A Unreal Engine usa funções para controlar o comportamento do jogo, como responder a entradas do jogador, atualizar a física e renderizar gráficos.
> 
> **Como usar C++ na Unreal Engine:**
> 
> 1.  **Criando Classes:** Você cria classes C++ no Unreal Editor usando o assistente "New C++ Class". Isso gera arquivos `.h` (cabeçalho) e `.cpp` (implementação) para sua classe.
> 2.  **Escrevendo Código:** No arquivo `.cpp`, você implementa as funções da sua classe. No arquivo `.h`, você declara as variáveis e funções que sua classe possui.
> 3.  **Compilando:** Depois de escrever o código, você precisa compilá-lo. O Unreal Editor faz isso automaticamente quando você salva os arquivos ou pode compilar manualmente.
> 4.  **Usando no Editor:** Depois de compilado, sua nova classe estará disponível no Unreal Editor. Você pode arrastá-la para o mundo do jogo, modificar suas propriedades e conectá-la a outros objetos.
> 
> **Exemplo de Código:**
> 
> ```cpp
> // MeuAtor.h
> #pragma once
> 
> #include "CoreMinimal.h"
> #include "GameFramework/Actor.h"
> #include "MeuAtor.generated.h"
> 
> UCLASS()
> class MEUPROJETO_API AMeuAtor : public AActor
> {
> 	GENERATED_BODY()
> 
> public:	
> 	// Sets default values for this actor's properties
> 	AMeuAtor();
> 
> protected:
> 	// Called when the game starts or when spawned
> 	virtual void BeginPlay() override;
> 
> public:	
> 	// Called every frame
> 	virtual void Tick(float DeltaTime) override;
> 
> 	//Componente de malha estática para exibir um modelo 3D
> 	UPROPERTY(VisibleAnywhere, BlueprintReadOnly, Category = "Componentes", meta = (AllowPrivateAccess = "true"))
> 	UStaticMeshComponent* MalhaEstatica;
> 
> };
> 
> // MeuAtor.cpp
> #include "MeuAtor.h"
> #include "Components/StaticMeshComponent.h"
> 
> // Sets default values
> AMeuAtor::AMeuAtor()
> {
>  	// Set this actor to call Tick() every frame.  You can turn this off to improve performance if you don't need it.
> 	PrimaryActorTick.bCanEverTick = true;
> 
> 	//Cria o componente de malha estática
> 	MalhaEstatica = CreateDefaultSubobject<UStaticMeshComponent>(TEXT("MalhaEstatica"));
> 	RootComponent = MalhaEstatica;
> }
> 
> // Called when the game starts or when spawned
> void AMeuAtor::BeginPlay()
> {
> 	Super::BeginPlay();
> 	
> }
> 
> // Called every frame
> void AMeuAtor::Tick(float DeltaTime)
> {
> 	Super::Tick(DeltaTime);
> 
> }
> ```
> 
> **Explicação do Código:**
> 
> *   `#include`: Inclui arquivos de cabeçalho necessários.
> *   `UCLASS()`: Macro que marca a classe como uma classe da Unreal Engine, permitindo que ela seja usada no Editor.
> *   `AMeuAtor : public AActor`: Declara uma classe chamada `AMeuAtor` que herda de `AActor`.
> *   `UPROPERTY()`: Macro que declara uma variável como uma propriedade da Unreal Engine, permitindo que ela seja editada no Editor.
> *   `UStaticMeshComponent* MalhaEstatica`: Ponteiro para um componente de malha estática, que exibirá um modelo 3D.
> *   `CreateDefaultSubobject<UStaticMeshComponent>(TEXT("MalhaEstatica"))`: Cria uma instância do componente de malha estática.
> *   `RootComponent = MalhaEstatica`: Define o componente de malha estática como o componente raiz do ator.
> *   `BeginPlay()`: Função que é chamada quando o jogo começa.
> *   `Tick()`: Função que é chamada a cada frame.
> 
> **Analogia:**
> 
> Pense em construir uma casa. A Unreal Engine fornece os tijolos, janelas e portas (classes e componentes). O C++ permite que você projete a planta da casa (crie novas classes), decida onde colocar cada cômodo (adicione componentes) e como as coisas vão funcionar (escreva funções).
> 
> Lembre-se, começar com C++ na Unreal pode parecer desafiador no início, mas com prática e paciência, você vai dominar as ferramentas e criar jogos incríveis!

------------------------------------------------------------
Você> Python: me explique a história do brasil?
Agente Tutor> Hmm, pensando em como te ajudar com python (explicar)...

--- Resposta do AgenteExplicador ---


> Desculpe, mas sou um agente especializado em programação e não tenho informações sobre a história do Brasil. Posso te ajudar com conceitos e dúvidas sobre a linguagem Python, se quiser.

------------------------------------------------------------
Você> sair

Agente Tutor> Chat encerrado. Bons estudos!

Agente Tutor> Chat encerrado. Continue explorando o mundo da programação!
